In [336]:
# 모듈 로딩
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [337]:
# 데이터 로딩
DATA_PATH = '../Dataset/팀원 데이터/Linear_model(2.9~3.1).csv'
dataDF = pd.read_csv(DATA_PATH)

### train_test 나누기

In [338]:
dataDF

,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
0,69.6,189,67.2,3.010000,67.1
1,69.8,189,67.2,3.010000,67.0
2,69.7,189,67.9,3.080000,65.9
3,69.7,189,67.8,3.080000,65.9
4,69.7,189,67.8,3.080000,65.9
...,...,...,...,...,...
139826,70.4,185,66.7,3.060000,66.9
139827,70.4,185,66.6,3.035316,66.8
139828,70.3,185,66.7,3.036060,66.7
139829,70.4,185,66.7,3.036124,66.6


In [339]:
target = dataDF['scale_pv']
feature = dataDF.drop(columns='scale_pv')

In [340]:
target

0         3.010000
1         3.010000
2         3.080000
3         3.080000
4         3.080000
            ...   
139826    3.060000
139827    3.035316
139828    3.036060
139829    3.036124
139830    3.050000
Name: scale_pv, Length: 139831, dtype: float64

In [341]:
feature

,c_temp_pv,k_rpm_pv,n_temp_pv,s_temp_pv
0,69.6,189,67.2,67.1
1,69.8,189,67.2,67.0
2,69.7,189,67.9,65.9
3,69.7,189,67.8,65.9
4,69.7,189,67.8,65.9
...,...,...,...,...
139826,70.4,185,66.7,66.9
139827,70.4,185,66.6,66.8
139828,70.3,185,66.7,66.7
139829,70.4,185,66.7,66.6


In [342]:
X_train, X_test, y_train, y_test = train_test_split(feature,target, test_size= 0.2, random_state=77)

In [343]:
# Bayesian Optimization을 위한 목적 함수 정의
def xgb_evaluate(max_depth, n_estimators, learning_rate, colsample_bytree, subsample, min_child_weight):
    params = {
        'max_depth': int(max_depth),  # 정수형으로 변환
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),  # 정수형으로 변환
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'min_child_weight': int(min_child_weight),  # 정수형으로 변환
        'objective': 'reg:squarederror',
        'random_state': 2,
        'n_jobs': 4
    }
    model = XGBRegressor(**params)
    cv_scores = cross_val_score(model, X_train.values, y_train, cv=3, scoring='neg_mean_squared_error')
    return -cv_scores.mean()  # MSE를 최소화해야 하므로 부호 반전

## MSE는 낮을수록 좋은 지표이므로, 부호를 반전하여 반환
## Bayesian Optimization은 목적 함수가 최대화하는 방향으로 작동하기 때문에, MSE의 부호를 반전하여 "최적화 대상 점수"로 사용


In [344]:
# Bayesian Optimization 설정
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds={
        'max_depth': (3, 7),
        'n_estimators': (500, 1000),
        'learning_rate': (0.03, 0.1),
        'colsample_bytree': (0.5, 0.7),
        'subsample': (0.5, 0.8),
        'min_child_weight': (1, 5),
    },
    random_state=2,
)

In [345]:
# 최적화 실행
# init_points는 초기 random으로 몇개의 값을 찾을것인지(베이지안 최적화는 기존 데이터를 기반으로 가우시안 프로세스 모델)
# n_iter는 추가로 조사할 값들을 몇개 찾을지 설정(어디를 샘플링하면 최대값을 잘 찾을 수 있을지를 판단)
optimizer.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | 0.0001672 | 0.5872    | 0.03181   | 5.199     | 2.741     | 710.2     | 0.5991    |
| 2         | 0.0001668 | 0.5409    | 0.07335   | 4.199     | 2.067     | 810.6     | 0.6587    |
| 3         | 0.0001697 | 0.5269    | 0.06595   | 3.738     | 4.141     | 927.0     | 0.6483    |
| 4         | 0.0001667 | 0.6693    | 0.03558   | 5.021     | 1.261     | 714.1     | 0.529     |
| 5         | 0.0001703 | 0.5254    | 0.07177   | 3.904     | 1.428     | 610.2     | 0.6049    |
| 6         | 0.0001725 | 0.6243    | 0.0328    | 3.512     | 4.496     | 628.9     | 0.5348    |
| 7         | 0.0001648 | 0.6535    | 0.06315   | 6.992     | 3.396     | 652.0     | 0.6657    |
| 8         | 0.0001654 | 0.6191    | 0.03602   | 6.618     | 1.118     | 621.9     | 0.6141    |
| 9         | 0.0001

In [346]:
# 최적의 하이퍼파라미터 추출
best_params = optimizer.max['params']

# 하이퍼파라미터 정수형 변환 (필요 시)
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_weight'] = int(best_params['min_child_weight'])

In [347]:
# 최종 모델 생성 및 학습
final_model = XGBRegressor(
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    colsample_bytree=best_params['colsample_bytree'],
    subsample=best_params['subsample'],
    min_child_weight=best_params['min_child_weight'],
    objective='reg:squarederror',
    random_state=2,
    n_jobs=4
)

final_model.fit(X_train.values, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6243010951061198,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.03279748289216662,
             max_delta_step=0, max_depth=3, min_child_weight=4, missing=nan,
             monotone_constraints='()', n_estimators=628, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=2, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5347680210762598,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [348]:
# 모델 저장
final_model.save_model('xgb_LM_2.9 ~ 3.1.json')

In [349]:
loaded_model = XGBRegressor()
loaded_model.load_model('xgb_LM_2.9 ~ 3.1.json')

# 로드한 모델로 예측
y_pred = loaded_model.predict(X_test)

y_pred = loaded_model.predict(X_test)
MAE = mean_absolute_error(y_test,y_pred)
MAPE = mean_absolute_percentage_error(y_test,y_pred)
Score = loaded_model.score(X_test,y_test)
print(f'MAE => {MAE}     MAPE => {MAPE*100}    R2 => {Score}')

MAE => 0.005096931885343425     MAPE => 0.16795085218657616    R2 => 0.11516747966895902
